In [4]:
import numpy as np

In [4]:
#Question 2
#loading training dataset
train_data=np.loadtxt('train-MLAssignment1.csv',delimiter=',',skiprows=1)

#loading testing dataset
test_data=np.loadtxt('test-MLAssignment1.csv',delimiter=',',skiprows=1)

In [7]:
#Separating features and labels
X_train=train_data[:,1:]
Y_train=train_data[:,0]

X_test=test_data[:,1:]
Y_test=test_data[:,0]

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()

In [21]:
#Scaling train and test datasets
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)

In [22]:
no_of_features=X_train_scaled.shape[1]

weights=np.zeros(no_of_features)
bias=0.0

In [24]:
#y(predicted y)=W.X+b
def predict_price(X,weights,bias):
    return np.dot(X,weights)+bias

In [26]:
#cost function=1/m(predicted-original)^2
def calculate_cost(X,y,weights,bias):
    m=X.shape[0]
    predictions=predict_price(X,weights,bias)
    cost=(1/(2*m))*np.sum((predictions-y)**2)
    return cost
    

In [27]:
def gradient_descent(X,y,weights,bias,learning_rate):
    m= X.shape[0]
    predictions=predict_price(X,weights,bias)
    #calculating gradients
    dW = (1 / m) * np.dot(X.T, (predictions - y))
    db = (1 / m) * np.sum(predictions - y)
    #updating weights and bias
    weights -= learning_rate * dW
    bias -= learning_rate * db
    return weights,bias

In [44]:
learning_rate=0.01
iterations=1000

# Training loop
for i in range(iterations):
    weights, bias = gradient_descent(X_train_scaled, Y_train, weights, bias, learning_rate)


In [45]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Predict on the test set
Y_test_predicted = predict_price(X_test_scaled, weights, bias)

# Calculate MSE for the test set
test_mse = mean_squared_error(Y_test, Y_test_predicted)
print(f"Test MSE: {test_mse}")


Test MSE: 26690568587.087746


In [5]:
#Question 3
from numpy.linalg import solve

x_data= np.array([0,2,4,6,8,10])
y_data= np.array([0, 2.90, 14.8, 39.6, 74.3, 119])

In [6]:
#Part 1
#C=inverse of A.B
A=np.vander(x_data,6)
interpolation=solve(A,y_data)

def interpolating_polynomial(x):
    C=interpolation
    return C[0] + C[1]*x + C[2]*x**2 + C[3]*x**3 + C[4]*x**4 + C[5]*x**5

In [7]:
#Part 2
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

x_reshaped_data=x_data.reshape(-1,1)
polynomial = PolynomialFeatures(degree=2)
x_polynomial = polynomial.fit_transform(x_reshaped_data)
model = LinearRegression().fit(x_polynomial, y_data)
intercept_regression = model.intercept_
coefficients_regression = model.coef_

def regression_polynomial(x):
    return intercept_regression + coefficients_regression[1]*x + coefficients_regression[2]*x**2

In [8]:
#Part 2
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

x_reshaped_data=x_data.reshape(-1,1)
polynomial = PolynomialFeatures(degree=2)
x_polynomial = polynomial.fit_transform(x_reshaped_data)
model = LinearRegression().fit(x_polynomial, y_data)
intercept_regression = model.intercept_
coefficients_regression = model.coef_

def regression_polynomial(x):
    return intercept_regression + coefficients_regression[1]*x + coefficients_regression[2]*x**2

In [9]:
# Calculating force estimates at 750 ft/sec for both models
velocity_to_estimate = 7.5  # 750 ft/sec in terms of given scale (100 ft/sec units)
force_interpolating = interpolating_polynomial(velocity_to_estimate)
force_regression = regression_polynomial(velocity_to_estimate)

# Displaying the results
print(f"Estimated force at 750 ft/sec using Interpolating Polynomial: {force_interpolating} (100 lb)")
print(f"Estimated force at 750 ft/sec using Quadratic Regression Polynomial: {force_regression} (100 lb)")

Estimated force at 750 ft/sec using Interpolating Polynomial: 4951.088541666712 (100 lb)
Estimated force at 750 ft/sec using Quadratic Regression Polynomial: 64.29308035714286 (100 lb)


In [11]:
#Differences of both approaches:
#Each approach has its own strengths, depending on whether the goal is precision at given points or generalizability across a range of values.
#Interpolating Polynomial is ideal when an exact match to each data point is required, but it can become unstable and unreliable for extrapolation.
#Quadratic Regression Polynomial provides a generalized trend that balances accuracy with stability, making it useful for predictions and minimizing overall error, though it sacrifices exactness at individual points.
#Results of both approaches:
#The interpolating polynomial provides a very high estimate due to overfitting and oscillations from its high degree, making it unreliable for extrapolation. In contrast, the quadratic regression polynomial captures the general trend, providing a more stable and reasonable estimate outside the data range.










In [5]:
#Question 4
#loading training dataset
train_data=np.loadtxt('trainlr-MLAssignment1.csv',delimiter=',',skiprows=1)

#loading testing dataset
test_data=np.loadtxt('testlr-MLAssignment1.csv',delimiter=',',skiprows=1)

In [15]:
# Extracting features and target labels from training data
X_train = train_data[:, :-1]  
y_train = train_data[:, -1]  

# Extracting features and target labels from testing data
X_test = test_data[:, :-1]    
y_test = test_data[:, -1]     


In [7]:
#Part1
#f(z)=1/1+e^-z
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


In [8]:
#J=-y.log(f(x))-(1-y)log(1-f(x))
def compute_loss(y, y_pred):
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

In [9]:
def gradient_descent(X, y, learning_rate=0.01, epochs=1000):
    # Initializing weights and bias
    weights = np.zeros(X.shape[1])  
    bias = 0
    m = len(y)
    
    for epoch in range(epochs):
        # Calculating predictions
        z = np.dot(X, weights) + bias
        y_pred = sigmoid(z)
        
        # Calculating gradients
        error = y_pred - y
        gradient_weights = np.dot(X.T, error) / m
        gradient_intercept = np.mean(error)
        
        # Updating weights and bias
        weights -= learning_rate * gradient_weights
        bias -= learning_rate * gradient_intercept
    
    return weights, bias


In [10]:
#training
weights, bias = gradient_descent(X_train, y_train)

In [11]:
def predict(X, weights, bias):
    return sigmoid(np.dot(X, weights) + bias) >= 0.5


In [12]:
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred) * 100

In [13]:
# Making predictions on the test data
y_pred_test = predict(X_test, weights, bias)

# Calculating accuracy on the test data
test_accuracy = accuracy(y_test, y_pred_test)
print(f"Linear model's accuracy on test data: {test_accuracy:.2f}%")


Linear model's accuracy on test data: 75.00%


In [21]:
#Part2
#Polynomial feature transformation
def add_polynomial_features(X, degree=2):
    from itertools import combinations_with_replacement
    poly_features = [X]
    for d in range(2, degree + 1):
        for comb in combinations_with_replacement(range(X.shape[1]), d):
            poly_feature = np.prod([X[:, i] for i in comb], axis=0)
            poly_features.append(poly_feature[:, np.newaxis])
    return np.hstack(poly_features)

In [22]:
#Transforming features 
X_train_poly = add_polynomial_features(X_train, degree=3)
X_test_poly = add_polynomial_features(X_test, degree=3)
#Training non-linear model
weights_poly, intercept_poly = gradient_descent(X_train_poly, y_train)
#Testing
y_pred_test_poly = predict(X_test_poly, weights_poly, intercept_poly)
test_accuracy_poly = accuracy(y_test, y_pred_test_poly)
print(f"Non-linear model's test accuracy: {test_accuracy_poly:.2f}%")

Non-linear model's test accuracy: 75.00%


In [25]:
#Comparison:
#The linear and non-linear logistic regression models achieved the same accuracy of 75%, indicating that the data may be linearly separable, or 
#the polynomial transformation in the non-linear model did not significantly improve the fit. The linear model is preferable due to its simplicity, 
#efficiency, and interpretability, while the non-linear model, though more flexible, did not offer additional benefits in this case.